In [14]:
import pandas as pd
import sklearn as sk
import numpy as np
from rdkit import Chem
from pathlib import Path

In [15]:
import glob
test_csvs = glob.glob("../data/test_*.csv")
tests = {Path(t).stem : pd.read_csv(t) for t in test_csvs}
tests.keys()

dict_keys(['test_mord3d', 'test_morgan', 'test_rdk', 'test_descriptors'])

In [16]:
train_csvs = glob.glob("../data/train_*.csv")
train = {Path(t).stem : pd.read_csv(t) for t in train_csvs}
train.keys()

dict_keys(['train_mord3d', 'train_centroid_distances', 'train_morgan', 'train_descriptors', 'train_crystals', 'train_distances', 'train_rdk'])

In [19]:
train['train_crystals']['is_centrosymmetric']

0         True
1         True
2        False
3         True
4        False
         ...  
13444     True
13445     True
13446     True
13447     True
13448     True
Name: is_centrosymmetric, Length: 13449, dtype: bool

In [38]:
train['train_descriptors'].head()

,identifiers,Unnamed: 0,name,ABC,ABCGG,nAcid,nBase,SpAbs_A,SpMax_A,SpDiam_A,...,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2,InchiKey,SMILES
0,COXSUE,11365,Optimised_Molecules_V3.3460,8.773011,8.632564,0,0,14.599164,2.244739,4.388162,...,185.062283,8.046186,219,11,54.0,58.0,4.833333,2.777778,UYSCRWBEPUAZRA-UHFFFAOYSA-N,CC(=O)NN=C(C)C1=NCCS1
1,TUWFAU,58476,Optimised_Molecules_V3.19496,10.813451,10.346393,0,0,18.488707,2.357869,4.715737,...,199.110947,7.111105,364,22,70.0,80.0,6.805556,3.583333,RYPSYCXCJSHDJX-UHFFFAOYSA-N,CC(C)N(C)c1ccc(C#N)cc1C#N
2,DUTHEG,31879,Optimised_Molecules_V3.5074,10.922841,9.584172,0,0,16.685332,2.436274,4.872549,...,191.080710,8.307857,271,22,74.0,87.0,5.666667,3.000000,PVRUUABKIQNQFL-UHFFFAOYSA-N,Cc1cc(=O)n2c(N)nc(C)nc2n1
3,AXUDED08,2885,Optimised_Molecules_V3.988,7.140017,6.873356,0,0,12.857279,2.214320,4.428639,...,136.052429,7.558468,121,11,44.0,48.0,3.722222,2.444444,QPJVMBTYPHYUOC-UHFFFAOYSA-N,COC(=O)c1ccccc1
4,ARAGUV,2660,Optimised_Molecules_V3.795,7.399237,7.308033,0,0,11.638727,2.249778,4.311276,...,139.074562,7.319714,125,8,46.0,49.0,4.333333,2.250000,LUFRABHJXNJTNZ-UHFFFAOYSA-N,CC(=O)Nc1cc(C)[nH]n1


In [39]:
X = train['train_descriptors']
X = X.drop(['identifiers','name', 'InchiKey', 'SMILES'], axis = 1)
X

,Unnamed: 0,ABC,ABCGG,nAcid,nBase,SpAbs_A,SpMax_A,SpDiam_A,SpAD_A,SpMAD_A,...,SRW10,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2
0,11365,8.773011,8.632564,0,0,14.599164,2.244739,4.388162,14.599164,1.216597,...,8.764834,54.650723,185.062283,8.046186,219,11,54.0,58.0,4.833333,2.777778
1,58476,10.813451,10.346393,0,0,18.488707,2.357869,4.715737,18.488707,1.232580,...,9.421492,46.551655,199.110947,7.111105,364,22,70.0,80.0,6.805556,3.583333
2,31879,10.922841,9.584172,0,0,16.685332,2.436274,4.872549,16.685332,1.191809,...,9.693445,46.214439,191.080710,8.307857,271,22,74.0,87.0,5.666667,3.000000
3,2885,7.140017,6.873356,0,0,12.857279,2.214320,4.428639,12.857279,1.285728,...,8.703009,38.792641,136.052429,7.558468,121,11,44.0,48.0,3.722222,2.444444
4,2660,7.399237,7.308033,0,0,11.638727,2.249778,4.311276,11.638727,1.163873,...,8.616314,52.270726,139.074562,7.319714,125,8,46.0,49.0,4.333333,2.250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13444,34720,11.589507,11.335446,0,0,18.593111,2.444247,4.716186,18.593111,1.239541,...,9.554852,62.912651,214.008667,12.588745,344,20,78.0,92.0,5.916667,3.277778
13445,58781,8.594671,6.996374,0,0,15.070075,2.071027,4.107492,15.070075,1.255840,...,8.272571,40.358042,168.151415,5.254732,206,13,50.0,51.0,3.611111,2.916667
13446,38371,8.065904,7.608752,1,0,13.246202,2.248763,4.497527,13.246202,1.204200,...,8.879472,40.495691,153.042593,8.502366,158,12,50.0,54.0,4.583333,2.527778
13447,45201,10.813451,9.387164,0,0,17.683997,2.402187,4.804373,17.683997,1.263143,...,9.576441,45.924017,186.079313,7.753305,283,21,72.0,84.0,5.055556,3.083333


In [ ]:
x = np.array(x)
x[]

In [56]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer, SimpleImputer
imp = IterativeImputer(max_iter=10, random_state=0)
imp = SimpleImputer(strategy="most_frequent")

imp.fit(X)

X = imp.transform(X)

In [47]:
%time
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(random_state=0)
Y = train['train_crystals']['is_centrosymmetric']

clf.fit(X, Y)

CPU times: user 9 µs, sys: 4 µs, total: 13 µs
Wall time: 963 µs


RandomForestClassifier(random_state=0)

In [48]:
from sklearn.metrics import f1_score

f1_score(clf.predict(X), Y, average = 'macro')

1.0

In [61]:
# %% Imports + Data loading
from traceback import clear_frames
import numpy as np
import matplotlib.pyplot as plt
import sklearn as sk
import pandas as pd

from sklearn.pipeline import Pipeline
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import f_regression, chi2
from sklearn.feature_selection import SelectKBest
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.kernel_ridge import KernelRidge
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error
from sklearn.svm import SVR

from sklearn.ensemble import RandomForestClassifier

feature_headers = headers = [*pd.read_csv('../data/train_descriptors.csv', nrows=1)]
features = pd.read_csv(
    '../data/train_descriptors.csv',
    usecols=[c for c in headers if not c in ['identifiers', 'Unnamed: 0', 'name', 'InchiKey', 'SMILES']]
)
data = pd.read_csv('../data/train_crystals.csv')
# %% Train / test splitting
target = data['is_centrosymmetric']

X_train, X_test, y_train, y_test = train_test_split(
    features, target, test_size=0.33, random_state=42)

y_train = y_train.to_numpy()

# %% Full model defn as pipeline
pclf = Pipeline([
    ('imputer', SimpleImputer(strategy='mean', verbose=1)),
    ('scaler', MinMaxScaler()),
    ('feature_sel', SelectKBest(chi2, k = 10)),
    ('fitting', RandomForestClassifier(random_state=0))
])
# %% Fitting
pclf.fit(X_train, y_train)

# %% Prediction
y_pred = pclf.predict(X_test)
print(f1_(y_test, y_pred))
# %% plotting
plt.plot(y_test, y_pred, 'r.')
plt.show()
# %% saving
#np.savetxt('./out/task_1_predictions.csv', y_pred)
# %%


/Users/tom/miniconda3/lib/python3.7/site-packages/sklearn/impute/_base.py:438: UserWarning: Deleting features without observed values: [1049 1050 1051 1052 1053 1054 1092 1095 1103 1104 1105 1106 1107 1108
 1109 1110 1111 1112 1113 1114 1115 1116 1117 1119 1120 1121 1122 1124
 1125 1126 1127 1128 1129 1130 1131 1132 1133 1171 1174 1182 1183 1184
 1185 1186 1187 1188 1189 1190 1191 1192 1193 1194 1195 1196 1198 1199
 1200 1201 1203 1204 1205 1206]
  "observed values: %s" % missing)
/Users/tom/miniconda3/lib/python3.7/site-packages/sklearn/impute/_base.py:438: UserWarning: Deleting features without observed values: [1049 1050 1051 1052 1053 1054 1092 1095 1103 1104 1105 1106 1107 1108
 1109 1110 1111 1112 1113 1114 1115 1116 1117 1119 1120 1121 1122 1124
 1125 1126 1127 1128 1129 1130 1131 1132 1133 1171 1174 1182 1183 1184
 1185 1186 1187 1188 1189 1190 1191 1192 1193 1194 1195 1196 1198 1199
 1200 1201 1203 1204 1205 1206]
  "observed values: %s" % missing)


TypeError: numpy boolean subtract, the `-` operator, is not supported, use the bitwise_xor, the `^` operator, or the logical_xor function instead.